# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [5]:
path = "Section3/lesson 28"

from utils import Session

session = Session("Lesson29")
spark = session.session

## Read in the dataFrame for this Notebook

In [7]:
airbnb = spark.read.csv(f'{path}/nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [8]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [9]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [16]:
from pyspark.sql.types import (
    IntegerType
)
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [17]:
df.count()

49079

### 2. How many total reviews does each host have?

In [35]:
df.groupBy(["host_id", "host_name"])\
    .agg(sum("number_of_reviews").alias("Sum reviews nro"))\
    .sort(desc("Sum reviews nro"))\
    .limit(10).toPandas()

,host_id,host_name,Sum reviews nro
0,37312959,Maya,2273
1,344035,Brooklyn& Breakfast -Len-,2205
2,26432133,Danielle,2017
3,35524316,Yasu & Akiko,1971
4,40176101,Brady,1818
5,4734398,Jj,1798
6,16677326,Alex And Zeena,1355
7,6885157,Randy,1346
8,219517861,Sonder (NYC),1281
9,23591164,Angela,1269


### 3. Show the min and max of all the numeric variables in the dataset

In [24]:
df.select([
    "price", 
    "minimum_nights", 
    "number_of_reviews", 
    "reviews_per_month", 
    "calculated_host_listings_count"])\
    .summary("min", "max").limit(10).toPandas()


,summary,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count
0,min,-74,0,0,0,0
1,max,10000,1250,629,58,365


### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [36]:
df.groupBy(["host_id","host_name"])\
    .agg(sum("number_of_reviews").alias("Sum reviews nro"))\
    .sort(desc("Sum reviews nro"))\
    .limit(1).toPandas()

,host_id,host_name,Sum reviews nro
0,37312959,Maya,2273


### 5. On average, how many nights did most hosts specify for a minimum?

In [28]:
df.agg(avg("minimum_nights")).toPandas()

,avg(minimum_nights)
0,7.128613


### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [34]:
df.groupBy(["neighbourhood", "neighbourhood_group"]).agg(avg("price").alias("Avg Price")).sort(desc("Avg Price")).limit(30).toPandas()

,neighbourhood,neighbourhood_group,Avg Price
0,Fort Wadsworth,Staten Island,800.000000
1,Woodrow,Staten Island,700.000000
2,Sea Gate,Brooklyn,548.333333
3,Tribeca,Manhattan,490.638418
4,Riverdale,Bronx,442.090909
5,Prince's Bay,Staten Island,409.500000
6,Battery Park City,Manhattan,367.557143
7,Randall Manor,Staten Island,352.944444
8,Flatiron District,Manhattan,341.925000
9,NoHo,Manhattan,297.855263


### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [39]:
df.filter("room_type in ('Private room', 'Shared room')")\
    .groupBy(["room_type"])\
    .pivot("neighbourhood_group", ["Manhattan", "Brooklyn"])\
    .agg(avg("price")).show()

+------------+------------------+-----------------+
|   room_type|         Manhattan|         Brooklyn|
+------------+------------------+-----------------+
| Shared room| 89.06903765690376|50.52784503631961|
|Private room|116.05400302114803|76.47234042553191|
+------------+------------------+-----------------+



### Alright that's all folks!

### Great job!